# Empirical Orthogonal Function Analysis

EOF analysis provides a simple and efficient means of evaluating the dominant modes of variability contained in a dataset. It is done by moving to a new mathematical basis which relates to the covariance of the data, and then focusing the analysis effort on the basis vectors that describe the majority of the variance. It is <u>Principal Component Analysis</u> applied to spatio-temporal data.

This code calculates both spatial and temporal eigenvectors (also known as eigenfunctions or EOFs) that describe the variance of a spatio-temporal dataset (e.g. shoreline position data along  a beach). 

The aim is to describe the datasets variability across both space and time with new basis functions. These are purely mathematic functions (eigenfunctions) but can sometimes relate closely to physical processes. Also, there is some research suggesting a relationship between alongshore and cross-shore transport processes and the shape of the eigenfunctions;
- extrema in the spatial eigenfunctions relates to areas of maximum variability,
- nodal points in the spatial eigenfunctions relates to areas of stability (low variability),
- multiple nodal points in the spatial eigenfunction may indicate the importance of longshore processes,
- spatial eigenfunctions without nodes can describe shoreline response to cross-shore processes (i.e. the entire coast advances/retreats in phase).

A full positive/negative spatial eigenfunction implies that if erosion (or accretion) is occurring at one location, erosion (or accretion) is also occurring at every other location. Gradients in this eigenfunction can imply a different rate of erosion/accretion.

See per Miller & Dean (2006, 2007): Shoreline variability via EOF analysis - https://doi.org/10.1016/j.coastaleng.2006.08.013.
This method has been used since by Harley et al. (2011) - https://doi.org/10.1029/2011JF001989 - to show that cross-shore processes are an important mode of variability in beach rotation.

Other useful references (EOFs or Principal Component Analysis):
- http://stockage.univ-brest.fr/~herbette/Data-Analysis/data_analysis_eof.pdf
- https://towardsdatascience.com/the-mathematics-behind-principal-component-analysis-fff2d7f4b643
- https://www.cygres.com/OcnPageE/Glosry/OcnEof1E.html

## 1 - Import Python Modules

In [ ]:
# import packages and modules
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
# System check
import sys
print('Python version:', sys.version,
      '\nAnaconda environment:', sys.executable)

## 2 - User Inputs

In [ ]:
# Set and Check Directory
dataDir = 'C:/Users/s5245653/OneDrive - Griffith University/Projects/NaturalShorelineVariability_Grassy/data/'
plotDir = 'Plots/All/EOFs/'
os.chdir(dataDir)

# User inputs
resamplePeriod = '1MS' # see Pandas documentation for frequency choices for resampling (e.g. monthly - '1MS')
transectSpacing = 50 # metres
inputFile = dataDir + 'RSP_raw.csv'
idxCol = 'dates' # index column in csv data (needs to be datetime)
minVarianceOfInterest = 10 # percent
firstDate = datetime.datetime(1987, 9, 1, tzinfo=datetime.timezone.utc) # year, month, day
lastDate = datetime.datetime(2020, 12, 31, tzinfo=datetime.timezone.utc)

# Plots
figSize = [10,6] # figure size in inches
resolution = 600 # dpi
yTitle = 'EOF' 
xTitle = 'Alongshore Distance From West (m)'
labelLoc = 'upper left'

yRange = [-0.55, 0.55] # Spatial EOF
yRangeTemp = [-0.199, 0.199] # Temporal EOF
plotName = dataDir + plotDir + f'{yTitle}_RSP.jpg'

# Second Plot
colTitles = ['Spatial', 'Temporal']
labelLetters = [['a)', 'b)'], ['c)', 'd)']]
plot2Name = dataDir + plotDir + f'{yTitle}_all_RSP.jpg'

## 3 - Load and Clean Data

In [ ]:
# Load Raw Data
df = pd.read_csv(inputFile, index_col = idxCol)
df.index = pd.to_datetime(df.index)

# Create a dataset clean enough for analysis (remove nan values etc.)
df2 = df.copy(deep = True)
df2 = df2.dropna() # EOF analysis doesn't work with NaN values
df2 = df2.truncate(before = firstDate, after = lastDate)
len(df2)

## 4 - Compute Eigenfunctions

In [ ]:
# Create matrix of the data
y = df2.to_numpy()
y_t = y.transpose()

# Create a spatial covariance-like matrix
temp = 1 / y.size
a = temp * np.matmul(y_t, y) # a measure of the spatial covariance
b = temp * np.matmul(y, y_t) # a measure of the temporal covariance

In [ ]:
# Compute eigenvalues and eigenvectors from matrices
eigenValues, eigenVectors = np.linalg.eig(a)
eigenValuesB, c = np.linalg.eig(b)

# Returns eigenValues in order from largest to smallest, with eigenVectors returned as corresponding columns
idx = eigenValues.argsort()[::-1]
eigenValues = eigenValues[idx]
e_x = eigenVectors[:,idx]

idx = eigenValuesB.argsort()[::-1]
eigenValuesB = eigenValuesB[idx] # first nx eigenvaluesB should equal eigenvalues from A
c_t = c[:,idx]

# Calculate percent of variance represented by the eigenfunctions
pOfVar = np.divide(100 * np.real(eigenValues), np.sum(np.real(eigenValues)))
k = sum(map(lambda x : x > minVarianceOfInterest, pOfVar)) # number of eigenfunctions (vectors) to plot
# Ensure at least 2 eigenfunctions are plotted
if k == 1:
    k = 2
    
# Weights
w = [np.sqrt(eigenValues[i]*y.size) for i in range(k)]

# Check eigenvalues match between spatial and temporal EOFs
np.round(np.real(eigenValuesB[0:min(y.shape)]), 4) == np.round(eigenValues, 4)

In [ ]:
for i in range(k):
    print(f'EOF{i+1}: {np.round(pOfVar[i], 2)}%')
    
print(f'Therefore, the first {k} EOFs describe {np.round(np.sum(pOfVar[0:k]),1)} of the total variance!')

## 5 - Plot Results

In [ ]:
# Plot Spatial variability

# Plot Variables
numTransects = len(e_x)
xtickLabels = df.columns
xticks = np.arange(0, numTransects)

# Plotting
fig, ax = plt.subplots(figsize = figSize, tight_layout = True)

for i in range(k):
    ax.plot(np.real(e_x[:,i]), label = f'Spatial EOF{i+1} {np.round(pOfVar[i], 2)}%')
ax.legend(loc = labelLoc)
ax.axhline(0, color = 'k')
#ax.grid()

plt.xticks(xticks, labels = xtickLabels, rotation = 45)
plt.xlabel(xTitle)
plt.ylabel(yTitle)
plt.ylim(yRange)
plt.autoscale(enable=True, axis='x', tight=True)

#plt.savefig(plotName, dpi = resolution)

In [ ]:
fig, axes = plt.subplots(nrows = k, ncols = 2, figsize = figSize, tight_layout = True)

for i in range(k):
    axes[i,0].plot(np.real(e_x[:,i]), label = f'{np.round(pOfVar[i], 2)}%')
    axes[i,0].set_ylabel(f'EOF {i+1}')
    axes[i,0].set_ylim(yRange)
    axes[i,0].legend(loc = labelLoc)
    axes[i,1].plot(df2.index, np.real(c_t[:,i]), label = f'{np.round(pOfVar[i], 2)}%')
    axes[i,1].set_ylim(yRangeTemp) 
    for j in range(2):
        axes[i,j].autoscale(enable = True, axis ='x', tight = True)
        axes[i,j].axhline(0, color = 'k')
        axes[i,j].tick_params(direction="in")
        axes[i,j].text(-0.1, 0.95, labelLetters[i][j], transform=axes[i,j].transAxes)
    axes[i,0].set_xticks([0,5,10,15,20,24])
    axes[i,0].set_xticklabels([0, 250, 500, 750, 1000, 1200])

# Spatial and Temporal Titles
for ax, col in zip(axes[0], colTitles):
    ax.set_title(col)
axes[1,0].set_xlabel(xTitle)
axes[1,1].set_xlabel('Date (yr)')

#plt.savefig(plot2Name, dpi = resolution)
plt.plot()

In [ ]:
c_t1 = np.real(c_t[:,0])
c_t2 = np.real(c_t[:,1])
c_t3 = np.real(c_t[:,2])

df2['c_t1'] = c_t1
df2['c_t2'] = c_t2
df2['c_t3'] = c_t3

In [ ]:
xtickMonths = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Monthly averages of "rotation" variability
fig, ax = plt.subplots(figsize = figSize, tight_layout = True)

ax.plot(df2.groupby(df2.index.month)['c_t2'].mean())
ax.axhline(0, color = 'k')
ax.tick_params(direction="in")
ax.autoscale(enable = True, axis ='x', tight = True)
plt.xticks(range(1,13), labels = xtickMonths, rotation = 45)
plt.xlabel('Month')
plt.ylabel('EOF2 Mean')
plt.ylim(-0.075, 0.075)
#plt.savefig(dataDir + plotDir + 'rotationEOF.jpg', dpi = resolution)
plt.show()

# The End

## Further Analysis with SAM

In [ ]:
SAMloc = 'C:/Users/s5245653/OneDrive - Griffith University/Projects/GrassyWaves/Data/SAMIndex.csv'
SOIloc = 'C:/Users/s5245653/OneDrive - Griffith University/Projects/GrassyWaves/Data/SOI_monthly.txt'


SAMI = pd.read_csv(SAMloc, index_col = 'Unnamed: 0', parse_dates = True)
SAMI.index = pd.to_datetime(SAMI.index, utc = True)
SAMI = SAMI.truncate(before = firstDate, after = lastDate)
SAMI.columns = ['SAMI']


SOI = pd.read_csv(SOIloc, header = None, index_col = 0, parse_dates = True)
SOI.index = pd.to_datetime(SOI.index, format = '%Y%m', utc = True)
SOI = SOI.truncate(before = firstDate, after = lastDate)
SOI.columns = ['SOI']

SAM_SOI = SAMI.join(SOI)
SAM_SOI = SAM_SOI.reset_index()
SAM_SOI.columns = ['Dates', 'SAMI', 'SOI']
SAM_SOI.head()

In [ ]:
df_ = df2.copy(deep = True)
df_ = df_.reset_index()


merged_df = pd.merge(
    df_,
    SAM_SOI[['SAMI', 'SOI']],
    left_on=[df_.dates.dt.month, df_.dates.dt.year],
    right_on=[SAM_SOI.Dates.dt.month, SAM_SOI.Dates.dt.year],
)
merged_df.index = merged_df.dates
merged_df = merged_df.drop(['key_0', 'key_1', 'dates'], axis = 1)
merged_df.head()

In [ ]:
SAM_winter = SAMI[(SAMI.index.month.isin([6,7,8]))]
SAM_winter = SAM_winter.resample('3MS').mean()
SAM_winter['winter_to_winter'] = SAM_winter.index.map(lambda x: x.year if x.month >= 1 else x.year-1)
SAMposyears = SAM_winter[SAM_winter['SAMI'] >= 0].winter_to_winter
SAMnegyears = SAM_winter[SAM_winter['SAMI'] < 0].winter_to_winter

merged_df['winteryear'] = merged_df.index.map(lambda x: x.year if x.month >= 1 else x.year-1)


df4 = merged_df[merged_df.winteryear.isin(SAMnegyears)]
df4 = df4.drop(['c_t1', 'c_t2', 'c_t3', 'SAMI', 'SOI', 'winteryear'], axis = 1)

In [ ]:
# Create matrix of the data
y4 = df4.to_numpy()
y4_t = y4.transpose()

# Create a spatial covariance-like matrix
temp = 1 / y4.size
a4 = temp * np.matmul(y4_t, y4) # a measure of the spatial covariance
b4 = temp * np.matmul(y4, y4_t) # a measure of the temporal covariance


# Compute eigenvalues and eigenvectors from matrices
eigenValues4, eigenVectors4 = np.linalg.eig(a4)
eigenValuesB4, c4 = np.linalg.eig(b4)

# Returns eigenValues in order from largest to smallest, with eigenVectors returned as corresponding columns
idx4 = eigenValues4.argsort()[::-1]
eigenValues4 = eigenValues4[idx4]
e_x4 = eigenVectors4[:,idx4]

idx4 = eigenValuesB4.argsort()[::-1]
eigenValuesB4 = eigenValuesB4[idx4] # first nx eigenvaluesB should equal eigenvalues from A
c_t4 = c4[:,idx4]

# Calculate percent of variance represented by the eigenfunctions
pOfVar4 = np.divide(100 * np.real(eigenValues4), np.sum(np.real(eigenValues4)))
k4 = sum(map(lambda x : x > minVarianceOfInterest, pOfVar4)) # number of eigenfunctions (vectors) to plot
# Ensure at least 2 eigenfunctions are plotted
if k4 == 1:
    k4 = 2
    
# Weights
w4 = [np.sqrt(eigenValues4[i]*y4.size) for i in range(k4)]


# Plot Spatial variability

# Plot Variables
numTransects = len(e_x4)
xtickLabels = df4.columns
xticks = np.arange(0, numTransects)

# Plotting
fig, ax = plt.subplots(figsize = figSize, tight_layout = True)

for i in range(k4):
    if i == 0:
        ax.plot(np.multiply(-1,np.real(e_x4[:,i])), label = f'Spatial EOF{i+1} {np.round(pOfVar4[i], 2)}%')
    else:
        ax.plot(np.real(e_x4[:,i]), label = f'Spatial EOF{i+1} {np.round(pOfVar4[i], 2)}%')
ax.legend(loc = labelLoc)
ax.axhline(0, color = 'k')
#ax.grid()

plt.xticks(xticks, labels = xtickLabels, rotation = 45)
plt.xlabel(xTitle)
plt.ylabel(yTitle)
plt.ylim(yRange)
plt.autoscale(enable=True, axis='x', tight=True)

#plotName = '_pos.jpg'
#plt.savefig(plotName, dpi = resolution)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(merged_df['SAMI'], merged_df['c_t2'])

In [ ]:
# Ranges of shoreline positions from the first EOF related to each transect location
# e.g. shoreline variability resulting from the first EOF has a range of 19 m at the western profile 
ranges = []
means = []
for j in range(k):
    temp = [np.ptp((e_x[:,j] * w[j])[i] * c_t1) for i in range(y.shape[1])]
    temp2 = [np.mean((e_x[:,j] * w[j])[i] * c_t1) for i in range(y.shape[1])]
    ranges.append(temp)
    means.append(temp2)

In [ ]:
pOfVar[0] + pOfVar[1]